## Alpha/Beta Diversity for EAC Progresson
Date: 5/15/23  
Goal: Get basic micorbial alpha/beta diversity  
Will likely expand upon this more before the paper, but for now just getting an idea of what our samples look like

### Imports

In [13]:
import pandas as pd
import qiime2 as q2
from qiime2 import Artifact, Metadata
from biom import load_table

from qiime2.plugins.taxa.visualizers import barplot
from qiime2.plugins.diversity.pipelines import alpha
from qiime2.plugins.diversity.visualizers import alpha_group_significance

In [2]:
#WOL Taxonomy in FeatureData[Taxonomy] Form
wol_taxonomy = Artifact.import_data('FeatureData[Taxonomy]', '/Users/cguccion/Dropbox/Storage/HelpfulLabDocs/taxonomy_trees/WOL/lineages.txt', 'HeaderlessTSVTaxonomyFormat')
wol_taxonomy

<artifact: FeatureData[Taxonomy] uuid: 83206f28-ab1f-47c7-bf20-92a7e6b9744b>

### Functions

In [3]:
def taxonomy_barplot(fn, taxonomy=wol_taxonomy):
    '''Create taxonomy barplots'''
    
    ft = Artifact.load('processed_data/qza/' + fn + '.qza')
    meta = q2.Metadata(pd.read_csv('processed_data/metadata/metadata_' + fn + '.tsv', sep='\t', index_col=0))
    
    barplot_output = barplot(table=ft, taxonomy=taxonomy, metadata=meta)
    barplot_output_v = barplot_output.visualization
    barplot_output_v.save('outputs/taxonomy_barplots/barplot_' + fn + '.qzv')

In [22]:
def basic_alpha(fn, metric = 'observed_features'):
    '''Run basic alpha diversity'''
    
    ft = Artifact.load('processed_data/qza/' + fn + '.qza')
    meta = q2.Metadata(pd.read_csv('processed_data/metadata/metadata_' + fn + '.tsv', sep='\t', index_col=0))
    
    alpha_vector, = alpha(table= ft, metric=metric)
    alpha_output = alpha_group_significance(alpha_diversity = alpha_vector, metadata = meta)
    alpha_output_v = alpha_output.visualization
    alpha_output_v.save('outputs/alpha_plots/alpha_' + fn + '.qzv')

### Taxonomy Barplots

In [6]:
taxonomy_barplot('normal_genome_WOL')

In [7]:
taxonomy_barplot('GERD_genome_WOL')

In [8]:
taxonomy_barplot('BE_genome_WOL')

In [9]:
taxonomy_barplot('EAC_genome_WOL')

In [4]:
taxonomy_barplot('BE_Ross_paired_genome_WOL')

In [5]:
taxonomy_barplot('EAC_Ross_paired_genome_WOL')

In [9]:
taxonomy_barplot('BE_Prog_T2_genome_WOL')

### Alpha Plots

In [24]:
basic_alpha('BE_Ross_paired_genome_WOL')

In [25]:
basic_alpha('EAC_Ross_paired_genome_WOL')

In [23]:
basic_alpha('BE_Prog_T2_genome_WOL')

In [26]:
basic_alpha('EAC_genome_WOL')